# selenium으로 진행

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import psycopg2
from itertools import product
from time import sleep
import datetime

In [2]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
BASE_URL = 'http://sbiznews.com/news/?menu=1&menuid=44&action=index'
# DETAIL_URL = 'https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq='
# https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq=4

In [4]:
browser.get(BASE_URL)

In [39]:
browser.switch_to.window(window_name=browser.window_handles[0])

In [40]:
table = browser.find_element(by=By.XPATH, value='/html/body/table/tbody/tr[1]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[2]/tbody/tr[2]/td/table/tbody')

In [41]:
rows = table.find_elements(by=By.TAG_NAME, value='tr')

In [44]:
today = datetime.datetime.now().strftime('%Y%m%d')
# today = '20220725'

In [52]:
crawl_list = []

for row in rows:
    tds = row.find_elements(by=By.TAG_NAME, value='td')
    tds.reverse() # 날짜부터 확인 가능하도록
    
    isToday = False
    for td in tds:
        if today == td.text.replace("[","").replace("]","").replace("/","").strip():
            isToday = True
        
        a_tag = td.find_elements(by=By.TAG_NAME, value='a')
        if isToday and len(a_tag) > 0:
            crawl_list.append(a_tag[0].get_attribute('href'))

In [54]:
for crawl_target in crawl_list:
    browser.get(crawl_target)

In [9]:
title = browser.find_element(by=By.CLASS_NAME, value='view_ltit').text
title

'국세청, 중소기업 4만여곳 대상 법인세 공제·감면 컨설팅'

In [10]:
contents = browser.find_element(by=By.ID, value='ContentsLayertext').text
contents

"[연합뉴스 제공] 국세청이 다음 달부터 중소기업 4만여곳 중 신청하는 기업에 대해 법인세 공제ㆍ감면 컨설팅을 진행한다고 26일 밝혔다.\n< 중소기업 법인세 공제·감면 컨설팅 >\n직전 사업연도 수입금액이 100억원 이상, 1천억원 미만인 중소기업이면 컨설팅을 신청할 수 있다.\n컨설팅은 기업이 고용ㆍ설비투자 등을 한 뒤 공제ㆍ감면 적용 여부와 금액 등을 홈택스나 관할 지방국세청을 통해 문의하면 국세청이 신청한 날의 다음 달 말일까지 서면으로 답변해주는 방식으로 진행된다.\n과거 사업연도에 공제ㆍ감면을 적용받지 못해 경정청구를 할 때는 청구 전 컨설팅 신청이 가능하다.\n중소기업이 컨설팅 내용에 따라 법인세를 신고하면 신고내용 확인과 감면사후관리 대상에서 제외되고 나중에 세무조사를 받아 컨설팅 내용과 다르게 과세처분 되더라도 과소신고 가산세를 면제받는다.\n국세청은 2020년 7월부터 일부 중소기업에 대해 세무컨설팅을 진행해왔으나, 기업이 컨설팅을 '세무간섭'으로 인식해 제도 활용이 적다는 지적이 있었다.\n국세청이 지난달 532개 중소기업을 대상으로 한 설문조사 결과에 따르면 62%의 기업은 세액공제ㆍ감면 적용에 가장 큰 어려움을 겪고 있다고 응답하기도 했다.\n이에 국세청은 전체 세목에 대해 진행하던 컨설팅을 세액공제ㆍ감면 위주로 개편하고 대상 중소기업도 협약을 맺은 일부 기업에서 신청하는 모든 중소기업으로 확대하기로 했다."

In [11]:
url = browser.current_url
url

'http://sbiznews.com/news/?action=view&menuid=44&no=52940&page=&skey=&sword='

In [12]:
def insert_news(params):
    
    sql = """INSERT INTO tb_crawling_news_intrf(id, title, contents, gb, reg_dt, url, interface_dt)
             VALUES((select coalesce(max(id),0)+1 from tb_crawling_news_intrf), %s, %s, %s, %s, %s, now());"""
    conn = None

    try:
        conn = psycopg2.connect(host='58.120.227.138', dbname='kt_jalnagage',user='postgres',password='Wjdrlwjd1!',port=5432)
        cur = conn.cursor()
        
        cur.execute(sql, params)
        conn.commit()
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [13]:
insert_news([title, contents, 'news', today, url])